# Coursework 2 Submission

### Student Number: 210590034

In [1]:
# import required libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.ml.feature import Bucketizer
from pyspark.sql import types
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression

In [2]:
# create spark session
sc = SparkSession.builder.appName("fraud").getOrCreate()
sc

In [3]:
# import .csv from hdfs
df_train = sc.read.csv('fraudTrain.csv', header=True, inferSchema = True)
df_test = sc.read.csv('fraudTest.csv', header=True, inferSchema = True)

In [4]:
#count number of rows of our dataframes
num_rows_train = df_train.count()
num_rows_test = df_test.count()
print(f'Number of rows in train: {num_rows_train}')
print(f'Number of rows in test: {num_rows_test}')

Number of rows in train: 1296675
Number of rows in test: 555719


In [5]:
# combining both train and test sets for data analysis
df = df_train.union(df_test)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: string (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [6]:
#count number of rows of our combined dataFrame
num_rows = df.count()
print(f'Number of rows: {num_rows}')

Number of rows: 1852394


In [7]:
#count the number of original data rows
df_row_count =df.count()
print("number of original data rows: ", df_row_count)
#count the number of data rows after deleting duplicated data
df_duplicate_count = df.dropDuplicates().count()
print("number of data rows after deleting duplicated data: ", df_duplicate_count)
differenece = df_row_count - df_duplicate_count
print("number of duplicate rows: ", differenece)

number of original data rows:  1852394
number of data rows after deleting duplicated data:  1852394
number of duplicate rows:  0


In [8]:
# use how="all" for missing data in the entire column
df_missing = df.dropDuplicates().dropna(how="any")
df_missing_any = df_row_count - df_missing.count()
print("number of rows with missing data: ", df_missing_any)

number of rows with missing data:  0


In [9]:
# combine 'first' and 'last' columns to create 'full_name column'
df = df.withColumn('full_name', sf.concat(sf.col('first'),sf.lit(' '), sf.col('last')))
df.show(1, vertical=True)

-RECORD 0-------------------------------------
 _c0                   | 0                    
 trans_date_trans_time | 2019-01-01 00:00:18  
 cc_num                | 2703186189652095     
 merchant              | fraud_Rippin, Kub... 
 category              | misc_net             
 amt                   | 4.97                 
 first                 | Jennifer             
 last                  | Banks                
 gender                | F                    
 street                | 561 Perry Cove       
 city                  | Moravian Falls       
 state                 | NC                   
 zip                   | 28654                
 lat                   | 36.0788              
 long                  | -81.1781             
 city_pop              | 3495                 
 job                   | Psychologist, cou... 
 dob                   | 1988-03-09           
 trans_num             | 0b242abb623afc578... 
 unix_time             | 1325376018           
 merch_lat   

In [10]:
# checking balance of fraud cases (target variable)
df.select("is_fraud").groupBy("is_fraud").count().show()
fraud_count = df.select("is_fraud").where(df.is_fraud == '1').count()
legit_count = df.select("is_fraud").where(df.is_fraud == '0').count()
print(f'Fraud percentage = {fraud_count / df_row_count}')
print(f'Legitamite percentage = {legit_count / df_row_count}')

+--------+-------+
|is_fraud|  count|
+--------+-------+
|       1|   9651|
|       0|1842743|
+--------+-------+

Fraud percentage = 0.005210014716091717
Legitamite percentage = 0.9947899852839083


Data is highly imbalanced

In [11]:
# number of unique categorical values
customers = df.select('full_name').distinct().count()
cards = df.select('cc_num').distinct().count()
merchant = df.select('merchant').distinct().count()
category = df.select('category').distinct().count()
street = df.select('street').distinct().count()
city = df.select('city').distinct().count()
state = df.select('state').distinct().count()
zip_codes = df.select('zip').distinct().count()
lat = df.select('lat').distinct().count()
long = df.select('long').distinct().count()
job = df.select('job').distinct().count()
trans_num = df.select('trans_num').distinct().count()
dob = df.select('dob').distinct().count()

print(f'Number of distinct customers: {customers}')
print(f'Number of distinct cards: {cards}')
print(f'Number of distinct merchants: {merchant}')
print(f'Number of distinct categories: {category}')
print(f'Number of distinct streets: {street}')
print(f'Number of distinct cities: {city}')
print(f'Number of distinct states: {state}')
print(f'Number of distinct zip codes: {zip_codes}')
print(f'Number of distinct latitudes: {lat}')
print(f'Number of distinct longitudes: {long}')
print(f'Number of distinct jobs: {job}')
print(f'Number of distinct transaction numbers: {trans_num}')
print(f'Number of distinct dates of birth: {dob}')

Number of distinct customers: 989
Number of distinct cards: 999
Number of distinct merchants: 693
Number of distinct categories: 14
Number of distinct streets: 999
Number of distinct cities: 906
Number of distinct states: 51
Number of distinct zip codes: 985
Number of distinct latitudes: 983
Number of distinct longitudes: 983
Number of distinct jobs: 497
Number of distinct transaction numbers: 1852394
Number of distinct dates of birth: 984


In [12]:
# checking balance of male and female customers
df.select("gender").groupBy("gender").count().show()
female_count = df.select("gender").where(df.gender == 'F').count()
male_count = df.select("gender").where(df.gender == 'M').count()
print(f'Female percentage = {female_count / df_row_count}')
print(f'Male percentage = {male_count / df_row_count}')

+------+-------+
|gender|  count|
+------+-------+
|     F|1014749|
|     M| 837645|
+------+-------+

Female percentage = 0.5478040848761117
Male percentage = 0.45219591512388835


In [13]:
# use 'trans_date_trans_time' column to create column with the only the required info from the time (month, day, hour)
df = df.withColumn("trans_date_trans_time", sf.to_timestamp("trans_date_trans_time", 'yyyy-MM-dd HH:mm:ss'))
df = df.withColumn('month', sf.month(df.trans_date_trans_time))
df = df.withColumn('day', sf.date_format(df.trans_date_trans_time, "d"))
df = df.withColumn('hour', sf.hour(df.trans_date_trans_time))

In [14]:
# caluclate customer age at the time of transaction
df = df.withColumn("dob", sf.to_timestamp("dob", 'yyyy-MM-dd'))
df = df.withColumn("age_at_transaction",sf.round(sf.months_between(sf.col("trans_date_trans_time"),sf.col("dob"))/sf.lit(12)))

In [15]:
# drop unnecessary columns
df = df.drop('_c0','first','last','street','zip','lat','long','dob','trans_num','merch_lat',
             'merch_long','unix_time','trans_date_trans_time')

In [16]:
# display dataframe to check changes
df.toPandas()

,cc_num,merchant,category,amt,gender,city,state,city_pop,job,is_fraud,full_name,month,day,hour,age_at_transaction
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,Moravian Falls,NC,3495,"Psychologist, counselling",0,Jennifer Banks,1,1,0,31.0
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,Orient,WA,149,Special educational needs teacher,0,Stephanie Gill,1,1,0,41.0
2,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,Malad City,ID,4154,Nature conservation officer,0,Edward Sanchez,1,1,0,57.0
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,Boulder,MT,1939,Patent attorney,0,Jeremy White,1,1,0,52.0
4,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,Doe Hill,VA,99,Dance movement psychotherapist,0,Tyler Garcia,1,1,0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,M,Luray,MO,519,Town planner,0,Michael Olson,12,31,23,55.0
1852390,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,M,Lake Jackson,TX,28739,Futures trader,0,Jose Vasquez,12,31,23,21.0
1852391,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,F,Burbank,WA,3684,Musician,0,Ann Lawson,12,31,23,39.0
1852392,4079773899158,fraud_Breitenberg LLC,travel,7.99,M,Mesa,ID,129,Cartographer,0,Eric Preston,12,31,23,55.0


In [17]:
#show statistic of the data we want
df.select('amt','city_pop','age_at_transaction').describe().show()

+-------+------------------+------------------+------------------+
|summary|               amt|          city_pop|age_at_transaction|
+-------+------------------+------------------+------------------+
|  count|           1852394|           1852394|           1852394|
|   mean| 70.06356747538618| 88643.67450931066| 46.26536201261719|
| stddev|159.25397477398303|301487.61834365036|17.412095278772394|
|    min|               1.0|                23|              14.0|
|    max|           28948.9|           2906700|              96.0|
+-------+------------------+------------------+------------------+



In [18]:
# create function to calculate quantiles for numerical variables
def get_quantile(column):
    quantile = df.approxQuantile([column], [0.25, 0.5, 0.75], 0)
    quantile_25 = quantile[0][0]
    quantile_50 = quantile[0][1]
    quantile_75 = quantile[0][2]
    print(f'quantiles for {column} column:')
    print('    quantile_25: '+str(quantile_25))
    print('    quantile_50: '+str(quantile_50))
    print('    quantile_75: '+str(quantile_75))

In [19]:
# use created function to get quantiles for amt, city_pop, and age_at_transaction columns
get_quantile('amt')
get_quantile('city_pop')
get_quantile('age_at_transaction')

quantiles for amt column:
    quantile_25: 9.64
    quantile_50: 47.45
    quantile_75: 83.1
quantiles for city_pop column:
    quantile_25: 741.0
    quantile_50: 2443.0
    quantile_75: 20328.0
quantiles for age_at_transaction column:
    quantile_25: 33.0
    quantile_50: 44.0
    quantile_75: 57.0


In [20]:
# create function to find fraud cases for each categorical variable
def top_fraud(column):
    df.groupBy(column).agg(sf.sum('is_fraud').alias("sum_fraud") \
                               ,sf.round(sf.avg('is_fraud'), 4).alias('avg_fraud')) \
                                .sort(sf.desc("avg_fraud")).show()

In [21]:
# correlation between merchant and fraud cases
top_fraud('merchant')

+--------------------+---------+---------+
|            merchant|sum_fraud|avg_fraud|
+--------------------+---------+---------+
|   fraud_Kozey-Boehm|       60|   0.0218|
|fraud_Herman, Tre...|       38|   0.0203|
|    fraud_Terry-Huel|       56|   0.0196|
|fraud_Kerluke-Abs...|       50|    0.019|
|fraud_Mosciski, Z...|       53|   0.0188|
|fraud_Schmeler, B...|       52|   0.0187|
|     fraud_Kuhic LLC|       53|   0.0186|
|      fraud_Jast Ltd|       51|   0.0185|
|fraud_Langworth, ...|       52|   0.0185|
|fraud_Boyer-Reichert|       51|   0.0184|
|fraud_Romaguera, ...|       51|   0.0184|
|fraud_Heathcote, ...|       51|   0.0183|
|   fraud_Goyette Inc|       50|    0.018|
| fraud_Lemke-Gutmann|       50|   0.0179|
|  fraud_Medhurst PLC|       48|   0.0175|
| fraud_Heathcote LLC|       47|   0.0169|
|  fraud_Rau and Sons|       60|   0.0169|
|fraud_Greenholt, ...|       46|   0.0168|
|fraud_Moore, Dibb...|       31|   0.0168|
|   fraud_Durgan-Auer|       31|   0.0168|
+----------

There appears to be no correlation between the merchant and the target variable

In [22]:
# correlation between transaction category and fraud cases
top_fraud('category')

+--------------+---------+---------+
|      category|sum_fraud|avg_fraud|
+--------------+---------+---------+
|  shopping_net|     2219|   0.0159|
|      misc_net|     1182|    0.013|
|   grocery_pos|     2228|   0.0126|
|  shopping_pos|     1056|   0.0063|
| gas_transport|      772|   0.0041|
|      misc_pos|      322|   0.0028|
|        travel|      156|   0.0027|
|   grocery_net|      175|   0.0027|
| personal_care|      290|   0.0022|
| entertainment|      292|   0.0022|
|     kids_pets|      304|   0.0019|
|   food_dining|      205|   0.0016|
|health_fitness|      185|   0.0015|
|          home|      265|   0.0015|
+--------------+---------+---------+



There appears to be a slight correlation between the transaction type and the target variable

In [23]:
# correlation between transaction month and fraud cases
top_fraud('month')

+-----+---------+---------+
|month|sum_fraud|avg_fraud|
+-----+---------+---------+
|    2|      853|   0.0087|
|    1|      849|   0.0081|
|    3|      938|   0.0065|
|    5|      935|   0.0064|
|   10|      838|   0.0061|
|    9|      758|   0.0054|
|    4|      678|    0.005|
|   11|      682|   0.0048|
|    6|      821|   0.0047|
|    8|      797|   0.0045|
|    7|      652|   0.0038|
|   12|      850|    0.003|
+-----+---------+---------+



There appears to be some correlation between the month and the number of fraud cases. As can be shown above, the fraud cases are appaarently I get to you

In [24]:
# correlation between transaction day of month and fraud cases
top_fraud('day')

+---+---------+---------+
|day|sum_fraud|avg_fraud|
+---+---------+---------+
| 12|      402|    0.007|
| 11|      397|   0.0069|
| 20|      419|   0.0069|
| 19|      364|   0.0063|
| 25|      369|   0.0063|
| 31|      217|    0.006|
|  3|      345|   0.0059|
|  8|      373|   0.0058|
|  4|      335|   0.0058|
| 17|      328|   0.0056|
|  2|      316|   0.0053|
| 10|      307|   0.0052|
| 28|      346|   0.0052|
|  7|      332|   0.0052|
| 14|      324|   0.0051|
| 13|      308|   0.0051|
|  5|      286|    0.005|
| 18|      285|    0.005|
| 23|      290|   0.0049|
| 27|      297|   0.0049|
+---+---------+---------+
only showing top 20 rows



There appears to be no significant correlations with the fraud cases

In [25]:
# correlation between transaction time and fraud cases
top_fraud('hour')

+----+---------+---------+
|hour|sum_fraud|avg_fraud|
+----+---------+---------+
|  22|     2481|    0.026|
|  23|     2442|   0.0255|
|   0|      823|   0.0136|
|   1|      827|   0.0135|
|   3|      803|   0.0132|
|   2|      793|    0.013|
|   5|       80|   0.0013|
|  18|      111|   0.0012|
|   7|       72|   0.0012|
|  20|       98|   0.0011|
|  14|      100|   0.0011|
|  21|      101|   0.0011|
|  19|      105|   0.0011|
|  15|      100|   0.0011|
|  13|       94|    0.001|
|  17|       94|    0.001|
|   8|       59|    0.001|
|   4|       61|    0.001|
|  16|       97|    0.001|
|  11|       59|    0.001|
+----+---------+---------+
only showing top 20 rows



There appears to be a correlation between the time of the day when the fraud cases. Fraud cases appear to be more prevalant during night time

In [26]:
# correlation between age of customer during transaction and fraud cases
top_fraud('age_at_transaction')

+------------------+---------+---------+
|age_at_transaction|sum_fraud|avg_fraud|
+------------------+---------+---------+
|              96.0|        7|    0.028|
|              77.0|       84|   0.0133|
|              87.0|       48|   0.0115|
|              92.0|       77|   0.0112|
|              18.0|       45|   0.0101|
|              63.0|      193|   0.0099|
|              79.0|      115|   0.0098|
|              71.0|      127|   0.0095|
|              78.0|       97|   0.0094|
|              86.0|       52|   0.0094|
|              56.0|      194|   0.0087|
|              57.0|      212|   0.0082|
|              58.0|      222|   0.0081|
|              80.0|       73|   0.0078|
|              76.0|       47|   0.0078|
|              59.0|      175|   0.0077|
|              62.0|      136|   0.0077|
|              89.0|       43|   0.0076|
|              53.0|      238|   0.0074|
|              85.0|       53|   0.0072|
+------------------+---------+---------+
only showing top

There appears to be a correlation between the age and the fraud cases appear to be good. As the favorite targets to fraud where, older and younger customers

In [27]:
# correlation between state the transaction happened in and fraud cases
top_fraud('state')

+-----+---------+---------+
|state|sum_fraud|avg_fraud|
+-----+---------+---------+
|   DE|        9|      1.0|
|   RI|       15|   0.0201|
|   AK|       50|   0.0169|
|   OR|      197|   0.0075|
|   NH|       79|   0.0067|
|   VA|      273|   0.0065|
|   TN|      159|   0.0064|
|   NE|      216|   0.0063|
|   MN|      280|   0.0062|
|   NY|      730|   0.0061|
|   DC|       31|    0.006|
|   KS|      193|   0.0059|
|   NV|       47|   0.0058|
|   CO|      115|   0.0058|
|   SC|      236|   0.0057|
|   ME|      134|   0.0057|
|   MS|      169|   0.0056|
|   IN|      217|   0.0055|
|   GA|      204|   0.0055|
|   WI|      228|   0.0055|
+-----+---------+---------+
only showing top 20 rows



There seems to be a strong correlation as the state 'DE' saw a 100% fraud cases

In [28]:
# correlation between city the transaction happened in and fraud cases
top_fraud('city')

+--------------+---------+---------+
|          city|sum_fraud|avg_fraud|
+--------------+---------+---------+
|         Wever|        6|      1.0|
|    Coulee Dam|       15|      1.0|
|       Norfolk|        7|      1.0|
|West Frankfort|       13|      1.0|
|    Wappapello|        8|      1.0|
|        Crouse|        8|      1.0|
|        Roland|       11|      1.0|
|        Waukau|       10|      1.0|
|       Seattle|       19|      1.0|
|       Ashland|       10|      1.0|
|   Springville|       12|      1.0|
|      Streator|        7|      1.0|
|    East China|        9|      1.0|
|        Angwin|       10|      1.0|
|   Chattanooga|        7|      1.0|
|         Craig|       14|      1.0|
|       Medford|        9|      1.0|
|     Senatobia|       10|      1.0|
|    Brookfield|        9|      1.0|
|   Orange Park|       10|      1.0|
+--------------+---------+---------+
only showing top 20 rows



There appears to be a strong correlation as a lot of the cities had a 100% fraud rate

In [29]:
# correlation between customer job and fraud cases
top_fraud('job')

+--------------------+---------+---------+
|                 job|sum_fraud|avg_fraud|
+--------------------+---------+---------+
| Information officer|        8|      1.0|
|Contracting civil...|        7|      1.0|
|Accountant, chart...|       11|      1.0|
|    Industrial buyer|       10|      1.0|
|  Veterinary surgeon|        8|      1.0|
|   Warehouse manager|        7|      1.0|
|              Dancer|       19|      1.0|
|   Personnel officer|       12|      1.0|
|Forest/woodland m...|        9|      1.0|
|      Engineer, site|       12|      1.0|
|Sales promotion a...|       14|      1.0|
|           Homeopath|       11|      1.0|
|   Software engineer|       11|      1.0|
|     Engineer, water|        8|      1.0|
|         Ship broker|        7|      1.0|
|Armed forces tech...|        8|      1.0|
|Broadcast journalist|        9|      1.0|
|Operational inves...|       11|      1.0|
|     Legal secretary|       12|      1.0|
|Air traffic contr...|       17|      1.0|
+----------

Same with the city column, job column seems to have a high correlation with the fraud cases.

In [30]:
# correlation between customer gender and fraud cases
top_fraud('gender')

+------+---------+---------+
|gender|sum_fraud|avg_fraud|
+------+---------+---------+
|     M|     4752|   0.0057|
|     F|     4899|   0.0048|
+------+---------+---------+



There appears to be no significant between gender and the fraud cases

In [31]:
# calculate correlation between amount of transaction and fraud cases
df.stat.corr('is_fraud','amt')

0.2093078472297254

In [32]:
# create transaction amount buckets
df = Bucketizer(splits=[-float('inf'), 0, 500, 1000, 1500, float('inf')],
                 inputCol='amt', outputCol='amt_bin'
        ).transform(df)

buckets = {1.0: "Less than 500", 2.0:"500 - 1000", 3.0: "1000 - 1500", 4.0: "More than 1500"}
udf_foo = sf.udf(lambda x: buckets[x], types.StringType())
df = df.withColumn("amt_bucket", udf_foo("amt_bin"))

# correlation between amount bucket and fraud cases
top_fraud('amt_bucket')

+--------------+---------+---------+
|    amt_bucket|sum_fraud|avg_fraud|
+--------------+---------+---------+
|   1000 - 1500|     1226|    0.339|
|    500 - 1000|     3458|   0.2136|
| Less than 500|     4967|   0.0027|
|More than 1500|        0|      0.0|
+--------------+---------+---------+



In [33]:
# define string columns to index
string_columns = ['category','city','state','job']

# Create an indexer that will create new columns with numeric index values
for col in string_columns:
    df = StringIndexer(inputCol=col, outputCol=f'{col}_idx').fit(df).transform(df)

In [34]:
# drop columns with low correlation with target variable
df = df.drop('cc_num','merchant','category','gender', 'city', 'state', 'city_pop','job','full_name','amt_bucket','amt_bin')
df.printSchema()

root
 |-- amt: double (nullable = true)
 |-- is_fraud: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- age_at_transaction: double (nullable = true)
 |-- category_idx: double (nullable = false)
 |-- city_idx: double (nullable = false)
 |-- state_idx: double (nullable = false)
 |-- job_idx: double (nullable = false)



In [35]:
# convert all column to integer
df_columns = df.columns
for column in df_columns:
    df = df.withColumn(column, df[column].cast(types.IntegerType()))

In [36]:
# Create an assembler object
assembler = VectorAssembler(inputCols=['amt', 'month', 'day', 'hour', 'age_at_transaction',
                                       'category_idx', 'city_idx', 'state_idx','job_idx'],
                            outputCol='features')

# Consolidate predictor columns
df_assembled = assembler.transform(df)

# Check the resulting column
df_assembled.select('features','is_fraud').show(5, truncate=False)

+---------------------------------------------+--------+
|features                                     |is_fraud|
+---------------------------------------------+--------+
|[4.0,1.0,1.0,0.0,31.0,11.0,173.0,12.0,129.0] |0       |
|[107.0,1.0,1.0,0.0,41.0,1.0,27.0,29.0,64.0]  |0       |
|[220.0,1.0,1.0,0.0,57.0,6.0,795.0,45.0,426.0]|0       |
|[45.0,1.0,1.0,0.0,52.0,0.0,661.0,39.0,205.0] |0       |
|[41.0,1.0,1.0,0.0,33.0,10.0,224.0,13.0,284.0]|0       |
+---------------------------------------------+--------+
only showing top 5 rows



In [37]:
# split data into train and test
df_train, df_test = df_assembled.randomSplit([0.8, 0.2], seed=100)

# # Check that training set has around 80% of records
training_ratio = df_train.count() / df_assembled.count()
print(f'Train dataframe to main dataframe ratio: {training_ratio}')

Train dataframe to main dataframe ratio: 0.8001586055666343


In [38]:
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier(featuresCol = 'features', labelCol='is_fraud')
tree_model = tree.fit(df_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(df_test)
prediction.select('is_fraud', 'prediction', 'probability').show(5, False)

+--------+----------+------------------------------------------+
|is_fraud|prediction|probability                               |
+--------+----------+------------------------------------------+
|0       |0.0       |[0.9963781971970396,0.0036218028029604303]|
|0       |0.0       |[0.9997172268656229,2.8277313437705625E-4]|
|0       |0.0       |[0.9997172268656229,2.8277313437705625E-4]|
|0       |0.0       |[0.9997172268656229,2.8277313437705625E-4]|
|0       |0.0       |[0.9997172268656229,2.8277313437705625E-4]|
+--------+----------+------------------------------------------+
only showing top 5 rows



In [39]:
prediction.groupBy('is_fraud', 'prediction').count().show()

# # Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND is_fraud = prediction').count()
TP = prediction.filter('prediction = 1 AND is_fraud = prediction').count()
FN = prediction.filter('prediction = 0 AND is_fraud = 1').count()
FP = prediction.filter('prediction = 1 AND is_fraud = 0').count()

# Calculate confusion matrix metrics
accuracy = (TN + TP) / (TN + TP + FN + FP)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)

# Print metrics
print(f"""
accuracy = {accuracy}
sensitivity = {sensitivity}
specificity = {specificity}
precision = {precision}
""" )

+--------+----------+------+
|is_fraud|prediction| count|
+--------+----------+------+
|       1|       0.0|  1010|
|       0|       0.0|368046|
|       1|       1.0|  1000|
|       0|       1.0|   129|
+--------+----------+------+


accuracy = 0.9969231600416009
sensitivity = 0.4975124378109453
specificity = 0.9996496231411692
precision = 0.8857395925597874



CART model performed well as it was able to identify close to 50% of the fraud cases

In [40]:
# Create classifier object and train on training data
logistic_model = LogisticRegression(featuresCol = 'features', labelCol='is_fraud').fit(df_train)

# Create a predictions for the test data and show confusion matrix
prediction = logistic_model.transform(df_test)
prediction.groupBy("is_fraud", "prediction").count().show()

+--------+----------+------+
|is_fraud|prediction| count|
+--------+----------+------+
|       1|       0.0|  2010|
|       0|       0.0|367992|
|       0|       1.0|   183|
+--------+----------+------+



In [41]:
# # Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND is_fraud = prediction').count()
TP = prediction.filter('prediction = 1 AND is_fraud = prediction').count()
FN = prediction.filter('prediction = 0 AND is_fraud = 1').count()
FP = prediction.filter('prediction = 1 AND is_fraud = 0').count()

# Calculate confusion matrix metrics
accuracy = (TN + TP) / (TN + TP + FN + FP)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)

# Print metrics
print(f"""
accuracy = {accuracy}
sensitivity = {sensitivity}
specificity = {specificity}
precision = {precision}
""" )


accuracy = 0.9940759350054702
sensitivity = 0.0
specificity = 0.9995029537584029
precision = 0.0



Logistic Regression model performed badly as it was not able to detect any of the fraud cases